# Pattern Matching e Lazy Evaluation

Conceitos a serem tratados:

1. Pattern Matching:
    * Conceito de atribuição via desestruturação;
        * A partir de Arrays;
        * A partir de objetos; 

2. Lazy Evaluation:
    * Definição do tipo "Lazy";
        * Variáveis;
        * Funções;
        * Estruturas de Dados;
            * Infinitas;
    * Computação análoga a lambda calculus;
        * Comparação com a sintaxe de lambda calculus

# Pattern Matching

## O que é Atribuição via Desestruturação?

A operação de atribuição é para salvar um valor em uma variável em tempo de execução, esta operação é da seguinte forma em typescript:

```
tres = 3;

lista = [1,2,3];
```

Existem momentos que é necessário atribuir mais de uma variável para valores salvos em uma estrutura de dados, como por exemplo:

```
let primeiroValor, segundoValor, restoDaLista;

primeiroValor = lista[0];

segundoValor = lista[1];

restoDaLista = lista.slice(2);
```

Vamos mostrar a sintaxe análoga em haskell:

```
separaLista lista = (primeiroValor, segundoValor, restoDaLista)
    where
        primeiroValor = lista !! 0
        segundoValor = lista !! 1
        restoDaLista = drop 2 lista
```

Utilizando o conceito de casamento de padrão em haskell, é possível escrever de forma reduzida:

```
separaLista lista = (primeiroValor, segundoValor, restoDaLista)
    where
        (primeiroValor: segundoValor: restoDaLista) = lista
```

O casamento de padrão neste caso ocorre da seguinte forma:
    * Temos os dois pontos representando uma operação para geração de uma lista;
    * Especificamos que a lista possui um primeiro valor, um segundo valor e dizemos que temos um resto da lista.

Em Javascript/Typescript, existe este conceito de casamento de padrão, ilustrado a seguir:

```
[primeiroValor, segundoValor, ...restoDaLista] = lista;
```

A atribuição está sendo realizada da seguinte forma, estamos atribuindo o primeiroValor como o primeiro valor da lista e assim por diante, estes "..." antes do restoDaLista apenas especifica que existe uma lista presente.

A seguir terá exemplos práticos com listas e com objetos

Analogia ao casamento de padrão em haskell

In [1]:
function separaLista(xs: number[]): [number, number[]] {
    let x, tailXs;
    [x, ...tailXs] = xs
    return [x, tailXs] 
}

Configuration is not found! Default configuration will be used: {"module":1,"target":1,"moduleResolution":2,"esModuleInterop":true}

In [2]:
separaLista([1,2,3])

[ 1, [ 2, 3 ] ]

In [3]:
function separaLista2(xs: number[]): [number, number, number[]] {
    let first, second, tail;
    [first, second, ...tail] = xs;
    return [first, second, tail];
}

undefined

In [4]:
separaLista2([1,2,3])

[ 1, 2, [ 3 ] ]

In [5]:
function trocaValores(xs: [number, number]): [number, number] {
    let [first, second] = xs;
    return [second, first];
}

undefined

In [6]:
trocaValores([1,2])

[ 2, 1 ]

### Desconstrução a partir de objetos

In [7]:
interface objeto {
    mensagem: string,
    descricao: string,
    tipo: string
}

undefined

In [8]:
const objetoBase: objeto = {
    "mensagem": "sou um objeto",
    "descricao": "nao tenho",
    "tipo": "json"
};

undefined

In [9]:
function pegaAtributos(objetoBase: objeto): [string, string, string] {
    let {mensagem, descricao, tipo} = objetoBase
    return [mensagem, descricao, tipo]
}

undefined

In [10]:
pegaAtributos(objetoBase)

[ 'sou um objeto', 'nao tenho', 'json' ]

In [11]:
function pegarAtributos2(objetoBase: objeto): [string, string] {
    let {mensagem, descricao} = objetoBase
    return [mensagem, descricao]
}

undefined

In [12]:
pegarAtributos2(objetoBase)

[ 'sou um objeto', 'nao tenho' ]

In [13]:
function pegarAtributos3(objetoBase: objeto): [string] {
    let {"mensagem": mensagem} = objetoBase;

    return [mensagem];
}

undefined

In [14]:
pegarAtributos3(objetoBase)

[ 'sou um objeto' ]

# Lazy Evaluation

definição de tipo lazy

In [15]:
type Lazy<T> = () => T | null

undefined

função lazy

In [16]:
function geraLazy<T>(x: T): Lazy<T> {
    return () => x;
}

undefined

In [17]:
let x = 10;

let variavelTeste = x;

let variavelTesteLazy = geraLazy(x);

console.log(`variavelTeste: ${variavelTeste}`);

console.log(`variavelTesteLazy: ${variavelTesteLazy}`);

console.log(`variavelTesteLazy avaliada: ${variavelTesteLazy()}`);


variavelTeste: 10
variavelTesteLazy: function () { return x; }
variavelTesteLazy avaliada: 10


undefined

In [18]:
function somaNormal(a: number, b: number): number {
    return a + b;
}

function somaLazy(a: Lazy<number>, b: Lazy<number>): Lazy<Number> {
    return geraLazy(a() + b());
}

undefined

In [19]:
let a = 1;
let b = 2;

let somaNormalValor = somaNormal(1,2);
let somaLazyValor = somaLazy(geraLazy(a), geraLazy(b));

console.log(`Soma normal: ${somaNormalValor}`);
console.log(`Soma lazy sem ser avaliada: ${somaLazyValor}`);
console.log(`Soma lazy sendo avaliada: ${somaLazyValor()}`);

Soma normal: 3
Soma lazy sem ser avaliada: function () { return x; }
Soma lazy sendo avaliada: 3


undefined

estrutura de dados lazy

In [20]:
type LazyList<T> = Lazy<{
    head: Lazy<T>,
    tail: LazyList<T>
}> | null

undefined

In [21]:
function geraLazyList<T>(lista: T[]): LazyList<T> {
    if (lista.length == 0) {
        return () => {
            return {
                head: null,
                tail: null
            }
        };

    }

    let [x, ...xs] = lista;

    return geraLazy({
        head: () => x,
        tail: geraLazyList(xs)
    });
}

undefined

In [22]:
var range = (i: Lazy<number>): LazyList<number> => {
    
    return () => { return {
        head: i,
        tail: range(() => i() + 1)
        }
    };
}

undefined

In [23]:
function toList<T>(lista: LazyList<T>): T[] {
    let listaAvaliada = lista();
    
    if (listaAvaliada.head == null) return [];

    let head = listaAvaliada.head;

    let tail = toList(listaAvaliada.tail);

    return [head(), ...tail];
}

undefined

In [24]:
function insertLazy<T>(lista: LazyList<T>, elemento: Lazy<T>): LazyList<T> {
    
    let listaAvaliada = lista();

    if (listaAvaliada.head == null) {
        
        listaAvaliada.head = elemento;
        listaAvaliada.tail = geraLazyList([]);
        
        return () => listaAvaliada;
    }

    return geraLazy({
        head: listaAvaliada.head,
        tail: insertLazy(listaAvaliada.tail, elemento)
    });
    
}

undefined

In [25]:
function containsLazy<T>(lista: LazyList<T>, elemento: Lazy<T>): boolean {

    let listaAvaliada = lista();

    if (listaAvaliada.head == null)
        return false;

    return listaAvaliada == elemento() || containsLazy(listaAvaliada.tail, elemento);
}

undefined

In [26]:
function removeLazy<T>(lista: LazyList<T>, elemento: Lazy<T>): LazyList<T> {

    let listaAvaliada = lista()

    if (listaAvaliada?.head() === elemento)
        return listaAvaliada.tail;

    return geraLazy({
        head: listaAvaliada?.head,
        tail: removeLazy(listaAvaliada?.tail, elemento)
    });

}

undefined

In [27]:
function takeLazy<T>(lista: LazyList<T>, i: number): LazyList<T> {
    
    if (i < 0 || lista().head === null)
        return geraLazyList([]);

    return () => {
        return {
            head: lista().head,
            tail: takeLazy(lista().tail, i-1)
        };
    }

}

undefined

In [28]:
function dropLazy<T>(lista: LazyList<T>, i: number): LazyList<T> {

    let listaAvaliada = lista();
    
    if (listaAvaliada.head === null)
        return geraLazyList([]);

    if (i == 0)
        return lista;

    return dropLazy(listaAvaliada.tail, i-1);

}

undefined

### Sandbox

In [29]:
var listaLazy = geraLazyList([1,2,3])

undefined

In [35]:
toList(dropLazy(listaLazy, 1))

[ 2, 3 ]

In [34]:
toList(takeLazy(range(() => 3), 99))

[
   3,   4,   5,   6,  7,  8,  9, 10, 11, 12, 13, 14,
  15,  16,  17,  18, 19, 20, 21, 22, 23, 24, 25, 26,
  27,  28,  29,  30, 31, 32, 33, 34, 35, 36, 37, 38,
  39,  40,  41,  42, 43, 44, 45, 46, 47, 48, 49, 50,
  51,  52,  53,  54, 55, 56, 57, 58, 59, 60, 61, 62,
  63,  64,  65,  66, 67, 68, 69, 70, 71, 72, 73, 74,
  75,  76,  77,  78, 79, 80, 81, 82, 83, 84, 85, 86,
  87,  88,  89,  90, 91, 92, 93, 94, 95, 96, 97, 98,
  99, 100, 101, 102
]

## Avaliação Lazy análoga a lambda calculus

In [31]:
function andLazy(a: Lazy<boolean>, b: Lazy<boolean>): Lazy<Boolean> {
    if (!a()) return geraLazy(false);
    return b;
}

undefined

In [32]:
function orLazy(a: Lazy<boolean>, b: Lazy<boolean>): Lazy<Boolean> {
    if (a()) return geraLazy(true);
    return b;
}

undefined

In [33]:
function xorLazy(a: Lazy<boolean>, b: Lazy<boolean>): Lazy<Boolean> {
    let avaliadoA = a();

    if (!a()) return geraLazy(false);

    return geraLazy(avaliadoA !== b());
}

undefined